In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from pandas.core.reshape.pivot import pivot
import numpy as np
import pandas as pd

import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import seaborn as sns
from IPython.display import clear_output

import random


# from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re

# from torch import optim
# import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [31]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [32]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import wandb

In [34]:
wandb.login(key='6264e29f109d3cf02a9911cd18b6725b79c489cd')

wandb: Currently logged in as: cs22m043. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [35]:
!pip install gdown
import gdown 
# !conda install -y gdown
import zipfile
url = 'https://drive.google.com/u/0/uc?id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw&export=download'  
gdown.download(url)
z= zipfile.ZipFile('aksharantar_sampled.zip')
z.extractall()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/u/0/uc?id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw&export=download
To: /content/aksharantar_sampled.zip
100%|██████████| 14.0M/14.0M [00:00<00:00, 159MB/s]


In [36]:
df = pd.read_csv("aksharantar_sampled/tel/tel_train.csv" , header = None)
# df.head()
df_val = pd.read_csv("aksharantar_sampled/tel/tel_valid.csv" , header = None)
df_test = pd.read_csv("aksharantar_sampled/tel/tel_test.csv" , header = None)
x1 = 0
english_words_val = df_val[x1]
english_words_test = df_test[x1]
english_words = df[x1]
telugu_words_val = df_val[x1+1]
telugu_words_test = df_test[x1+1]
telugu_words = df[x1+1]
#print(df[50000:])

In [37]:
def findCharList(eng_words,tel_words):
  english_char_list = []
  max_length_word_english = -1
  telugu_char_list = []
  max_length_word_telugu = -1
  for word in eng_words:
    x2 = len(word)
    x3 = max(max_length_word_english,x2)
    max_length_word_english = x3
    for char in word :
      c = char
      english_char_list.append(c);
  english_char_list = set(english_char_list)
  english_char_list = list(english_char_list)
  for word in tel_words:
    x4 = len(word)
    x5 = max(max_length_word_telugu,x4)
    max_length_word_telugu = x5
    for char in word :
      c1 =char
      telugu_char_list.append(c1);
  telugu_char_list = set(telugu_char_list)
  telugu_char_list = list(telugu_char_list)
  english_char_list.sort()
  telugu_char_list.sort()
  # print(len(telugu_char_list))
  #print(telugu_char_list)
  print(len(telugu_char_list))
  print(len(english_char_list))
  return english_char_list,telugu_char_list,max_length_word_english,max_length_word_telugu

In [38]:
english_char_list,telugu_char_list,max_length_word_english,max_length_word_telugu=findCharList(english_words,telugu_words)
print(english_char_list)
print(telugu_char_list)
print(max_length_word_telugu)
print(max_length_word_english)

62
26
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['ం', 'ః', 'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ', 'క', 'ఖ', 'గ', 'ఘ', 'చ', 'ఛ', 'జ', 'ఝ', 'ఞ', 'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల', 'ళ', 'వ', 'శ', 'ష', 'స', 'హ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ె', 'ే', 'ై', 'ొ', 'ో', 'ౌ', '్']
21
28


In [39]:
# for one word.
def english_word2vec(word):
  vec = []
  x6 = len(english_char_list)+1
  vec.append(x6)
  for char in word:
    c = char
    x7 = len(english_char_list)
    for i in range(x7):
      x8 = english_char_list[i]
      if(x8 == c):
        vec.append(i+1)
  x9 = max_length_word_english + 1
  while(len(vec) < x9): 
      x10 = 0
      vec.append(x10)
  x11 = 0
  vec.append(x11)
  return (vec)

In [40]:
def telugu_word2vec(word):
  vec = []
  x6 = len(telugu_char_list)+1
  vec.append(x6)
  for char in word:
    c = char
    x7 = len(telugu_char_list)
    for i in range(x7):
      x8 = telugu_char_list[i]
      if(x8 == c):
        vec.append(i+1)
  x9 = max_length_word_telugu + 1
  while(len(vec) < x9):  
      x10 = 0
      vec.append(x10)
  x11 = 0
  vec.append(x11)
  return(vec)

In [41]:
vec = telugu_word2vec(telugu_words[10])
print(telugu_words[10])
print(vec)
print("dheeraj")

వైఖరిలోను
[63, 45, 58, 17, 41, 51, 43, 60, 34, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dheeraj


In [42]:
vec = english_word2vec(english_words[10])
print(english_words[10])
print(vec)
print("varma")

vaikhariloonu
[27, 22, 1, 9, 11, 8, 1, 18, 9, 12, 15, 15, 14, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
varma


In [43]:
pred = []
gt=[]
inp = []

In [44]:
# creating matrix of representation for whole words of english and telugu.

def ip_matrix_construct(words, lang):
  ans = []
  if(lang == "english"):
    for word in words:
      ans.append(english_word2vec(word))
  else:
    for word in words:
      ans.append(telugu_word2vec(word))
  return(ans)

In [45]:
def matrixRep(english_words,telugu_words,english_words_val,telugu_words_val,english_words_test,telugu_words_test):
  # calculated representations of whole english and marathi words in variables english and marathi matrix.
  eng = "english"
  english_matrix = ip_matrix_construct(english_words, eng)
  
  english_matrix_val = ip_matrix_construct(english_words_val, eng)
  english_matrix_test = ip_matrix_construct(english_words_test, eng)
  
  
  english_matrix = torch.tensor(english_matrix)
  english_matrix_val = torch.tensor(english_matrix_val)
  english_matrix_test = torch.tensor(english_matrix_test)
  
  tel = "telugu"
  telugu_matrix = ip_matrix_construct(telugu_words, tel)
 
  telugu_matrix_val = ip_matrix_construct(telugu_words_val, tel)
  telugu_matrix_test =ip_matrix_construct(telugu_words_test, tel)

  telugu_matrix = torch.tensor(telugu_matrix) 
  telugu_matrix_val = torch.tensor(telugu_matrix_val)
  telugu_matrix_test = torch.tensor(telugu_matrix_test)
  
  return english_matrix,telugu_matrix,english_matrix_val,telugu_matrix_val,english_matrix_test,telugu_matrix_test

In [46]:
english_matrix,telugu_matrix,english_matrix_val,telugu_matrix_val,english_matrix_test,telugu_matrix_test = matrixRep(english_words,telugu_words,english_words_val,telugu_words_val,english_words_test,telugu_words_test)

In [47]:
class Encoder(nn.Module):
  def __init__(self,input_size, embedding_size, hidden_size, num_layers, dropout, cell_type, bidirectional,s=0.1):
    super(Encoder,self).__init__()
    ct = cell_type
    self.cell_type = ct
    dp = dropout
    self.dropout = nn.Dropout(dp)
    bd = bidirectional
    self.bidirectional = bd
    hs = hidden_size
    self.hidden_size = hs
    iS = input_size 
    self.embedding = nn.Embedding(iS, embedding_size)
    nl = num_layers
    self.num_layers = nl
    s1 = s
    self.s = s1
    if(ct == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = dropout, bidirectional = bidirectional)
    elif(cell_type == "RNN"):
      self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout = dropout, bidirectional = bidirectional)
    elif(ct == "GRU"):
      s1 = s1*2
      self.gru = nn.GRU(embedding_size, hidden_size, num_layers, dropout = dropout, bidirectional = bidirectional)

  def forward(self, x):
    k = x
    embedding = self.dropout(self.embedding(k))
    ct1 = self.cell_type
    if(ct1 == "LSTM"):
      outputs, (hidden,cell) = self.lstm(embedding)
      return hidden, cell
    elif(ct1 == "GRU"):
      output, hidden = self.gru(embedding)
      return output, hidden
    elif(ct1 == "RNN"):
      output, hidden = self.rnn(embedding)
      return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [48]:
class Decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout, cell_type,s=0.1):
    super(Decoder, self).__init__()
    ct = cell_type
    self.cell_type = ct
    hs = hidden_size
    self.hidden_size = hs
    dp = dropout
    self.dropout = nn.Dropout(dp)
    iS = input_size
    self.embedding = nn.Embedding(iS, embedding_size)
    self.s = s
    nl = num_layers
    self.num_layers = nl
   
    if(ct == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hs, nl, dropout = dp)
      es=embedding_size
    elif(ct == "GRU"):
      self.gru = nn.GRU(embedding_size, hs, nl, dropout = dp)
    elif(ct == "RNN"):
      self.rnn = nn.RNN(embedding_size, hs, nl, dropout = dp)
      s=s*2
    self.fc = nn.Linear(hs, output_size)  # fully connected.
  
  def forward(self, x, hidden, cell = 0):
    y = x
    y = y.unsqueeze(0).int()
    embedding = self.dropout(self.embedding(y))
    ct1 = self.cell_type

    if(ct1 == "LSTM"):
        outputs, (hidden, cell) = self.lstm(embedding, (hidden, cell))
    elif(ct1 == "RNN"):
        outputs, hidden = self.rnn(embedding, hidden)
    elif(ct1 == "GRU"):
        outputs, hidden = self.gru(embedding, hidden)
 
    predictions = self.fc(outputs)
    predictions = predictions.squeeze(0)
    if(ct1 == "LSTM"):
        return predictions, hidden, cell
    else:
        return predictions, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [49]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type, bidirectional, num_layers,s=0.1):
        super(Seq2Seq, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        nl = num_layers
        self.num_layers = nl
        bd = bidirectional
        self.bidirectional = bd
        ct = cell_type
        self.s = s
        self.cell_type = ct
        

    def forward(self, source, target, teacher_force_ratio=0.5,p=0.1):
        t2 = target.shape[0]
        target_len = t2
        s2 = source.shape[1]
        batch_size = s2
        tcl = len(telugu_char_list)
        target_vocab_size = tcl + 2  # intitially = 66 || Now 63 + 1(start token) + 1 = 65
        tvs = target_vocab_size
        outputs = torch.zeros(t2, s2, tvs).to(device)
        ct3 = self.cell_type
        if(ct3 == "GRU"):
            output, hidden = self.encoder(source)
        elif(ct3 == "RNN"):
            output, hidden = self.encoder(source)
        else:
            hidden, cell = self.encoder(source)
        bd3 = self.bidirectional 
        sl3 = self.num_layers
        if(bd3 == True):
            hidden = hidden[sl3 - 1] + hidden[sl3 - 1]
            hidden = hidden.repeat(sl3,1,1)
            if(self.cell_type == "LSTM"):
                cell = cell[sl3 - 1] + cell[sl3 - 1]
                cell = cell.repeat(sl3,1,1)
                
        x = target[0]
        s5=0.2
        for t in range(t2-1):
            if(ct3 == "LSTM"):
                output, hidden, cell = self.decoder(x, hidden, cell)
                s5=s5*2
            else :
                output, hidden = self.decoder(x, hidden)
                s5=s5+2

            outputs[t+1] = output
            best_guess = output.argmax(1)
            y = output
            x = target[t+1] if random.random() < teacher_force_ratio else best_guess

        return outputs


In [50]:
# def findDecoderInputSize():
#   x = len(telugu_char_list)
#   return x

In [51]:
# def findEncoderInputSize():
#   x = len(english_char_list)
#   return x

In [52]:
def Evaluate(model, english_matrix, telugu_matrix, epoch, batch_size,s=0.1):
    correct_count = 0
    bat = (int)(len(english_matrix) / batch_size)
    for batch_idx in range(1,bat+1):
        inp_data = english_matrix[batch_size * (batch_idx-1) : batch_size * (batch_idx)].to(device)
        id = inp_data
        bs = batch_size
        target = telugu_matrix[bs * (batch_idx-1) : bs * (batch_idx)].to(device)
        tgt = target
        # print(len(target))
        # print(type(target))
        # print("dheeraj1")
        # print(target[0])
        output = model.forward(id.T, tgt.T, 0)
        s1=s
        # print(output[0])
        # print("varma1")
        # print(target[0])
        output = nn.Softmax(dim=2)(output)
        #s1=s1*bs
        output = torch.argmax(output, dim=2)
        ot = output
        output = ot.T

        for i in range(1,bs+1):
            if(torch.equal(output[i-1][1:],target[i-1][1:])):
                k = correct_count
                correct_count = k+1
    l5 = len(english_matrix)
    accuracy = correct_count * 100 / l5
    return accuracy
        

In [56]:
def to_words(model, english_matrix, telugu_matrix, batch_size):
  bs=batch_size
  count=0
  for batch_idx in range((int)(len(english_matrix) / bs)):
        inp_data = english_matrix[bs * batch_idx : bs * (batch_idx+1)].to(device)
        inp_data1=inp_data
        target = telugu_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        output = model.forward(inp_data.T, target.T, 0)
        count=count+bs
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2)
        gt1=[]
        output = output.T
        count=count/(batch_idx+1)
        for i in range(len(inp_data)):
          inp_word = inp_data[i]
          word = ""
          for j in range(len(inp_word)):
            eng_char_array_size=27
            if(inp_word[j]>0 and inp_word[j]<eng_char_array_size):
              word += english_char_list[inp_word[j]-1]
          #temp1 = [word]
          #if(data_type == "test"):
          inp.append(word)
        for i in range(len(target)):
          word1 = ""
          word2 = ""
          target_word = target[i]
          output_word = output[i]
          gt1.append(output_word)
          for j in range(len(target_word)):
            if(target_word[j]>0 and target_word[j]<63):
              word1 += telugu_char_list[target_word[j] - 1]
          count1=count
          for j in range(len(output_word)):
            if(output_word[j]>0 and output_word[j]<63):
              word2 = word2 + telugu_char_list[output_word[j] - 1]
          temp = [word1, word2]
          
          gt.append(temp[0])
          pred.append(temp[1])
        
  df_test = pd.DataFrame({"Input": inp, "Ground Truth" : gt, "Model output":pred})
  df_test.to_csv("prediction_vanilla.csv", index=False)
            

In [54]:
def neural_network(cell_type, bidirectional, num_layers, batch_size, embedding_size, hidden_size, dropout,num_epochs=5,learning_rate=0.001):
  x=len(english_char_list)
  y=len(telugu_char_list)
  input_size_encoder = x + 2   
  input_size_decoder = y + 2  
  output_size        = y + 2  
  s=0.2
  encoder_net = Encoder(input_size_encoder, embedding_size, hidden_size, num_layers, dropout, cell_type,bidirectional).to(device)
  s=s+1
  decoder_net = Decoder(input_size_decoder,embedding_size,hidden_size,output_size,num_layers,dropout, cell_type).to(device)
  pad_idx = 63
  model = Seq2Seq(encoder_net, decoder_net, cell_type, bidirectional, num_layers).to(device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  
  criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

  # train_loss = []
  # train_accuracy = []
  # val_accuracy = []
  for epoch in range(1,num_epochs+1):
      ep = epoch-1
      print("epoch = ",ep)

      model.train()
      step = 0
      total_loss = 0
      bat = (int)(len(english_matrix) / batch_size)
      for batch_idx in range(1,bat+1):
          
          bi = batch_idx
          inp_data = english_matrix[batch_size * (bi-1) : batch_size * (bi)].to(device)
          target = telugu_matrix[batch_size * (bi-1) : batch_size * (bi)].to(device)

          target = target.T
          output = model(inp_data.T, target)
          step1 = step
          output = output[1:].reshape(-1, output.shape[2])
          s=0
          target = target[1:].reshape(-1)
          optimizer.zero_grad()
          loss = criterion(output, target)
          l1 = loss
          total_loss = total_loss + loss
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
          optimizer.step()
          s=s+1
          step = step + 1
  #     print("total loss = ",total_loss / (len(english_matrix) * len(marathi_matrix[0])))
      tl = total_loss/step
      print("total loss = ",tl)
      #train_loss.append(tl)

      # training_accuracy = Evaluate(model, english_matrix, telugu_matrix, ep, batch_size)
      # print("Training Accuracy = ", training_accuracy)
      # s+=1
      # #train_accuracy.append(training_accuracy)
      # val_accuracy = Evaluate(model, english_matrix_val, telugu_matrix_val, ep, batch_size)
      # print("Validation accuracy = ",val_accuracy)
      # test_accuracy = Evaluate(model, english_matrix_test, telugu_matrix_test, ep, batch_size)
      # print("Test accuracy = ",test_accuracy)
      #wandb.log({"train_accuracy": training_accuracy, "validation_accuracy": val_accuracy, "training_loss": total_loss / step,  'epoch': ep})
  #return train_loss,train_accuracy,val_accuracy
  test_accuracy = Evaluate(model, english_matrix_test, telugu_matrix_test, num_epochs, batch_size)
  print("Test accuracy = ",test_accuracy)
  to_words(model, english_matrix_test, telugu_matrix_test, batch_size)

In [57]:
#best hyperparameters
cell_type = "LSTM"
bidirectional = True
num_layers = 2
batch_size = 128
embedding_size = 128
hidden_size = 512
dropout = 0.4
num_epochs = 15
learning_rate = 0.001
neural_network(cell_type, bidirectional, num_layers, batch_size, embedding_size, hidden_size, dropout, num_epochs, learning_rate)

epoch =  0
total loss =  tensor(0.8946, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  1
total loss =  tensor(0.3339, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  2
total loss =  tensor(0.2266, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  3
total loss =  tensor(0.1810, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  4
total loss =  tensor(0.1537, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  5
total loss =  tensor(0.1298, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  6
total loss =  tensor(0.1160, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  7
total loss =  tensor(0.1041, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  8
total loss =  tensor(0.0905, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  9
total loss =  tensor(0.0811, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  10
total loss =  tensor(0.0745, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  11
total loss =  tensor(0.0656, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  12
total loss =  

In [35]:
#print(pvt[0][1])
# print(pvt[1][0])
# print(pvt[1][1])

వితనాన్ని


In [58]:
df_test = pd.read_csv("prediction_vanilla.csv")
df_test = df_test.sample(frac=1)
df_test.head(20)

,Input,Ground Truth,Model output
1385,coachella,కోచెల్లా,కాచెళ్ల
2253,digajaarchukuntunna,దిగజార్చుకుంటున్న,దిగజార్చుకుంటున్న
2343,ahaaramtoo,ఆహారంతో,ఆహారంతో
2659,kar,కర్,కర్
1122,cheppe,చెప్పే,చెప్పే
2222,aakaaraalu,ఆకారాలు,ఆకారాలు
1346,fleming,ఫ్లెమింగ్,ఫ్లెమింగ్
668,dial,డయల్,డియాల్
1188,rakshanhagaa,రక్షణగా,రక్షణగా
3460,bhavincheevaru,భావించేవారు,భావించేవారు


In [ ]:
# sweep_config = {
#     "method": 'bayes', #grid, random
#     "metric": {
#       "name": 'validation_accuracy',
#       "goal": 'maximize', 
#     },
#     "parameters": {
#         "epochs": {
#             "values": [15]
#         },
#         "dropout": {
#             "values": [0.5]
#         },
#         "hidden_size": {
#             "values": [256]
#         },
#         "embedding_size": {
#             "values": [128]
#         },
#         "learning_rate":{
#             "values":[0.001]
#         },
#         "batch_size":{
#             "values":[128]
#         },
#         "num_layers":{
#             "values":[2]
#        },
#         "bidirectional":{
#             "values":[False]
#         },
#         "cell_type":{
#             "values":['LSTM']
#         }
#     }
# }

In [ ]:
# sweep_config = {
#     "method": 'bayes', #grid, random
#     "metric": {
#       "name": 'validation_accuracy',
#       "goal": 'maximize', 
#     },
#     "parameters": {
#         "epochs": {
#             "values": [15]
#         },
#         "dropout": {
#             "values": [0.3,0.4,0.5]
#         },
#         "hidden_size": {
#             "values": [128,256,512]
#         },
#         "embedding_size": {
#             "values": [128,256,512]
#         },
#         "learning_rate":{
#             "values":[0.001,0.05]
#         },
#         "batch_size":{
#             "values":[32,64,128,256]
#         },
#         "num_layers":{
#             "values":[1,3,2]
#        },
#         "bidirectional":{
#             "values":[False,True]
#         },
#         "cell_type":{
#             "values":['LSTM','RNN','GRU']
#         }
#     }
# }

In [ ]:
# def train():
   
#     wandb.init()

#     #wandb.init()

#     # Config is a variable that holds and saves hyperparameters and inputs
#     config = wandb.config

#     name="ct_"+str(wandb.config.cell_type)+"_bd_"+str(wandb.config.bidirectional)+"_nl_"+str(wandb.config.num_layers)+"_bs_"+str(wandb.config.batch_size)+"_es_"+str(wandb.config.embedding_size)+"_hs_"+str(wandb.config.hidden_size)+"_dp_"+str(wandb.config.dropout)
#     print(name)
#     wandb.run.name = name
   
#     neural_network(cell_type=config.cell_type, bidirectional=config.bidirectional, num_layers=config.num_layers,batch_size = config.batch_size,embedding_size=config.embedding_size,hidden_size=config.hidden_size,dropout=config.dropout,num_epochs=config.epochs,learning_rate=config.learning_rate)

  

#     # for zi in range(len(train_acc)):
#     #     print(zi)        
#     #     wandb.log({'train_accuracy':train_acc[zi]})
#     #     wandb.log({'val_accuracy':val_acc[zi]})
#     #     wandb.log({'train_loss':train_loss[zi]}) 
#     #     #wandb.log({'val_loss':val_loss[zi]})
#     #     wandb.log({'num_epochs':zi+1})

#     wandb.finish()

In [ ]:
# sweep_id = wandb.sweep(sweep_config, entity="cs22m043", project="Assignment3")
# wandb.agent(sweep_id, train, count=5)

Create sweep with ID: nqjwyyj7
Sweep URL: https://wandb.ai/cs22m043/Assignment3/sweeps/nqjwyyj7


wandb: Agent Starting Run: nfyf87hy with config:
wandb: 	batch_size: 128
wandb: 	bidirectional: False
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.5
wandb: 	embedding_size: 128
wandb: 	epochs: 15
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 2
wandb: Currently logged in as: cs22m043. Use `wandb login --relogin` to force relogin


ct_LSTM_bd_False_nl_2_bs_128_es_128_hs_256_dp_0.5
epoch =  0
total loss =  tensor(1.5998, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  0.0
Validation accuracy =  0.0
epoch =  1
total loss =  tensor(1.1064, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  1.662109375
Validation accuracy =  4.150390625
epoch =  2
total loss =  tensor(0.6458, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  9.802734375
Validation accuracy =  15.8447265625
epoch =  3
total loss =  tensor(0.4578, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  18.763671875
Validation accuracy =  24.462890625
epoch =  4
total loss =  tensor(0.3707, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  24.75390625
Validation accuracy =  27.587890625
epoch =  5
total loss =  tensor(0.3165, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  31.138671875
Validation accuracy =  33.837890625
epoch =  6
total loss =  tensor(0.2800, device='cuda:0', grad_fn=

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_accuracy,▁▁▂▃▄▅▆▆▆▇▇▇███
training_loss,█▆▃▂▂▂▂▁▁▁▁▁▁▁▁
validation_accuracy,▁▂▃▅▅▆▆▇▇▇▇████
epoch,14
train_accuracy,53.38086
training_loss,0.16245
validation_accuracy,45.87402


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 9o9c69sn with config:
wandb: 	batch_size: 128
wandb: 	bidirectional: False
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.5
wandb: 	embedding_size: 128
wandb: 	epochs: 15
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 2


ct_LSTM_bd_False_nl_2_bs_128_es_128_hs_256_dp_0.5
epoch =  0


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
# id = wandb.sweep(sweep_config, project="Assignment3", entity ="cs22m043")
# wandb.agent(id, function=train)